In [4]:
import geopandas as gpd
import pandas as pd
import geemap
import ee
import shapely

In [5]:
ee.Authenticate()
ee.Initialize()

Planificación General: Medir área total de la laguna a lo largo del tiempo -> hacer un área total del uso de suelo usando aquella red neuronal que clasificaba segun crop o algo asi -> Ver como avanzó crop e industrial a lo largo del tiempo.

### Definimos como aoi un área de aproximadamente 5km alrededor de la Laguna de Aculeo, y el intervalo de tiempo de interés 2014-2024 ya que se secó alrededor de 2018, Además de la disponibilidad de COPERNICUS

In [40]:
doi = ["2012-01-01","2024-01-01"]
doi_intervalar = []
for i in range(14,25):
    doi_intervalar.append(str(2000+i)+"-01-01")
doi_intervalar

['2014-01-01',
 '2015-01-01',
 '2016-01-01',
 '2017-01-01',
 '2018-01-01',
 '2019-01-01',
 '2020-01-01',
 '2021-01-01',
 '2022-01-01',
 '2023-01-01',
 '2024-01-01']

In [24]:
# aoi = gpd.GeoSeries(shapely.Point(),crs="EPSG:4326").to_crs("EPSG:32719")
aoi = ee.Geometry.Point([-70.91725911612781,-33.84565081436696]).buffer(5000)
# aoi_bbox = ee.Geometry.Polygon(json.loads(aoi)['features'][0]['geometry']['coordinates'], 'EPSG:4326')

m = geemap.Map(center=[-33.84565081436696, -70.91725911612781], zoom=15)

m.addLayer(aoi,name='aoi')
m

Map(center=[-33.84565081436696, -70.91725911612781], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
Map = geemap.Map(center=[-33.84565081436696, -70.91725911612781],zoom=12)
for i in range(len(doi_intervalar)-1):
    pre_flood_start_date = doi_intervalar[i]
    pre_flood_end_date = doi_intervalar[i+1]

    # planificacion: sacar las imagenes -> geomediana -> calcular ndwi -> agregarlo a un array o algo así -> Hacer el gif

    Map.addLayer(este_año.clip(aoi), {'min': -25, 'max': -5}, 'SAR pre' + str(i))


Map